In [1]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModel


In [2]:
model_name = "Twitter/twhin-bert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of BertModel were not initialized from the model checkpoint at Twitter/twhin-bert-base and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
# Function to encode a single sentence and return its embeddings
def encode_argument(sentence):
    input_ids = tokenizer.encode(sentence, add_special_tokens=True, truncation=True, max_length=512, padding='max_length')
    input_ids_tensor = torch.tensor(input_ids).unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        outputs = model(input_ids_tensor)
        argument_embedding = outputs.last_hidden_state.mean(dim=1).squeeze(0)  # Mean pooling of token embeddings
    return argument_embedding.tolist()


In [ ]:
# Load the arguments from the .parquet file
arguments_df = pd.read_parquet("arguments.parquet")



In [ ]:
# Create embeddings for each argument and store them in a new column
arguments_df["embedding"] = arguments_df["argument"].apply(encode_argument)



In [ ]:
# Save the DataFrame with embeddings to a new .parquet file
arguments_df.to_parquet("arguments_with_embeddings.parquet", index=False)